การวิเคราะห์ความรู้สึก (Sentiment) ชอบ (Positive = 1) ไม่ชอบ (Negative = 0)

In [127]:
# pip install --upgrade gensim 
# pip install beautifulsoup4

In [128]:
import numpy as np 
import pandas as pd 
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, GRU, LSTM, Dense

gensim. __version__, np. __version__

('4.1.2', '1.22.1')

เตรียม Pre-trained model คำศัพท์

In [129]:
import gensim.downloader as api 

wv = api.load('glove-wiki-gigaword-100')
wv 

In [130]:
#ใช้ len ตรวจดูว่ามีคำกี่คำ
len(wv.index_to_key)

400000

In [131]:
#ตรวจดูขนาดของเวกเตอร์
wv.vector_size

100

ตรวจดูคำที่มีความคล้ายกัน เช่น horrible, terrible เพราะเกี่ยวกันการวิจารณ์ภาพยนต์จะมีคำบ่นต่างๆ เข้ามาเกี่ยวข้อง

In [132]:
#ตรวจดูคำที่มีความคล้ายกัน เช่น horrible, terrible เพราะเกี่ยวกันการวิจารณ์ภาพยนต์จะมีคำบ่นต่างๆ เข้ามาเกี่ยวข้อง
wv.similar_by_word('horrible')

[('awful', 0.921971321105957),
 ('terrible', 0.919477105140686),
 ('horrendous', 0.8366580605506897),
 ('dreadful', 0.8184822797775269),
 ('horrific', 0.8176026344299316),
 ('horrifying', 0.8023055791854858),
 ('appalling', 0.7695969939231873),
 ('tragic', 0.7407631874084473),
 ('disgusting', 0.7368147373199463),
 ('ugly', 0.7360007762908936)]

In [133]:
#หาคำที่มีความคล้ายกันมากที่สุด
wv.most_similar('crap', topn=8)

[('shit', 0.6642228364944458),
 ('fucking', 0.6362317800521851),
 ('whack', 0.6277898550033569),
 ('baloney', 0.5797702074050903),
 ('fuck', 0.5778754353523254),
 ('heck', 0.5700293779373169),
 ('damn', 0.5665410161018372),
 ('craziness', 0.5653018355369568)]

การจัดเตรียมข้อมูล

In [134]:
df = pd.read_csv('IMDB Dataset.csv', encoding='utf-8')
df.shape

(50000, 2)

In [135]:
df 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


สร้าง label เพื่อนำไปใช้ Train Test โดยให้ positive = 1 และ negative = 0

In [136]:
df['label'] = np.where(df.sentiment == 'positive',1,0)
df.sample(6)

,review,sentiment,label
40527,"If you've ever heard the saying, ""the book is ...",negative,0
35547,There's a certain irony in a parody of the Got...,negative,0
37664,"This is a beautiful, yet simple movie about on...",positive,1
6166,"Going into a movie like I this, I was expectin...",negative,0
16315,"Seriously engaging, intelligent and thought pr...",positive,1
46539,One of the many silent comedies Stan Laurel fe...,negative,0


การประมวลผลเบื้องต้น ได้แก่ การเปลี่ยนอักขระตัวเล็ก ตัดแท็ก HTML ตัดคำ ที่ไม่มีผลต่อความหมายของประโยค review 

In [137]:
#การเปลี่ยนคำให้เป็นอักษรตัวเล็ก

df['review2'] = df.review.str.lower()
df 

,review,sentiment,label,review2
0,One of the other reviewers has mentioned that ...,positive,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,1,a wonderful little production. <br /><br />the...
2,I thought this was a wonderful way to spend ti...,positive,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,0,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,"petter mattei's ""love in the time of money"" is..."
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,1,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,0,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,0,i'm going to have to disagree with the previou...


การใช้ index เพื่อตรวจดูอักษรที่มีแท็ก HTML ที่จะต้องตัดออกด้วย BeautifulSoup

In [138]:
#ตัดแท็ก HTML ออก
from bs4 import BeautifulSoup

def remove_tag(x):
    soup = BeautifulSoup(x)
    return soup.get_text()

df['review2'] = df.review2.apply(remove_tag)
df 

,review,sentiment,label,review2
0,One of the other reviewers has mentioned that ...,positive,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,1,a wonderful little production. the filming tec...
2,I thought this was a wonderful way to spend ti...,positive,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,0,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,"petter mattei's ""love in the time of money"" is..."
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,1,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,0,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,0,i'm going to have to disagree with the previou...


ใช้ remove_stopwords ตัดคำ เขียนทับลงไปในคอลัมน์ review2

In [139]:
from gensim.parsing.preprocessing import remove_stopwords
df['review2'] = df.review2.apply(remove_tag)
df 

,review,sentiment,label,review2
0,One of the other reviewers has mentioned that ...,positive,1,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,1,a wonderful little production. the filming tec...
2,I thought this was a wonderful way to spend ti...,positive,1,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,0,basically there's a family where a little boy ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1,"petter mattei's ""love in the time of money"" is..."
...,...,...,...,...
49995,I thought this movie did a down right good job...,positive,1,i thought this movie did a down right good job...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,0,i am a catholic taught in parochial elementary...
49998,I'm going to have to disagree with the previou...,negative,0,i'm going to have to disagree with the previou...


ตัดแบ่งคำ (Tokenize) และกรอกอักขระที่ไม่มีผล เช่น $%&^+-/*@<>= เป็นต้น 

In [140]:
#ตัดแบ่งคำ 
tk = Tokenizer()
tk.fit_on_texts(df.review2)

In [141]:
#ตรวจสอบคำที่ตัดแล้วพร้อมระบุความถี่ 
tk.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'movie': 16,
 'but': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'are': 22,
 'his': 23,
 'have': 24,
 'be': 25,
 'one': 26,
 'he': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'an': 31,
 'they': 32,
 'so': 33,
 'who': 34,
 'from': 35,
 'like': 36,
 'or': 37,
 'just': 38,
 'her': 39,
 'about': 40,
 'out': 41,
 'if': 42,
 "it's": 43,
 'has': 44,
 'there': 45,
 'some': 46,
 'what': 47,
 'good': 48,
 'when': 49,
 'more': 50,
 'very': 51,
 'up': 52,
 'no': 53,
 'time': 54,
 'my': 55,
 'even': 56,
 'would': 57,
 'she': 58,
 'which': 59,
 'only': 60,
 'really': 61,
 'see': 62,
 'story': 63,
 'their': 64,
 'had': 65,
 'can': 66,
 'me': 67,
 'well': 68,
 'were': 69,
 'than': 70,
 'much': 71,
 'we': 72,
 'bad': 73,
 'been': 74,
 'get': 75,
 'do': 76,
 'great': 77,
 'other': 78,
 'will': 79,
 'also': 80,
 'into': 81,
 'people': 82,

In [142]:
#ต้องการดูว่า film มีค่ารหัส index คืออะไร 
tk.word_index['film']

18

In [143]:
#หาจำนวนคำที่ไม่ซ้ำกัน (Unique)
vocab_size = len(tk.word_index)+1
vocab_size

126512

In [144]:
#เปลี่ยนรหัสคำให้เป็นรหัสตัวเลขในคอลัมน์ review2
sents_enc = tk.texts_to_sequences(df.review2)
sents_enc[1][:15]

[3, 392, 119, 351, 1, 1384, 2976, 6, 51, 17947, 51, 154, 54, 2379, 1583]

วิเคราะห์หาความยาวเฉลี่ย Min Max โดยนับความยาวของ list แต่ละ review จะได้ความยาวของ review ไม่เท่ากัน

In [145]:
sents_len = [len(i) for i in sents_enc]

print('Average Review Length:', np.mean(sents_len))
print('Standard Deviation:', round(np.std(sents_len)))
print('Min Length:', np.min(sents_len))
print('Max Length:', np.max(sents_len))

Average Review Length: 230.93744
Standard Deviation: 171
Min Length: 6
Max Length: 2466


การเตรียมข้อมูลก่อนการ Train จะต้องมีการเติม 0 (Padding) เพื่อให้แต่ละ review มีความยาวเท่ากัน ในที่นี่กำหนดเป็น 300 จากความยาวเฉลี่ย 231 (Average Review Length = 230.93744)

In [146]:
#จำนวนความยาวคำสูงสุด
max_len = 300
sents_pad = pad_sequences(sents_enc, truncating='post', padding='post', maxlen=max_len)
sents_pad

array([[  26,    4,    1, ..., 1601,   42,   21],
       [   3,  392,  119, ...,    0,    0,    0],
       [   9,  189,   10, ...,    0,    0,    0],
       ...,
       [   9,  234,    3, ...,    0,    0,    0],
       [ 144,  165,    5, ...,    0,    0,    0],
       [  53,   26, 5889, ...,    0,    0,    0]])

ถอดรหัส (Decode) ดูข้อความ Review ของ 2 คนแรกเขียนว่าอย่างไร 

In [147]:
check_reviews = sents_pad[:2]
tk.sequences_to_texts(check_reviews)

["one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda em city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare forget prett

การใช้งาน Tokenize ในการตัดคำและเข้ารหัสจะได้ออบเจกต์ tk (Tokenizer) เป็นค่าสำคัญต่างๆ เอาไว้ ซึ่งเวลานำ model ไปใช้ predict จะต้องใช้ออบเจกต์เข้ารหัสด้วยข้อความ review ที่นำมา predict 

In [148]:
#บันทึก Tokenizer object ไว้ใช้งานร่วมกับ model 
import pickle

with open('IMDB_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tk, handle, protocol=pickle.HIGHEST_PROTOCOL)

เมื่อนำ model ไปใช้งาน จะต้องโหลด tk (Tokenize) ดังนี้ 

In [149]:
#เมื่อนำ model ไปใช้งาน จะต้องโหลด tk (Tokenize) ดังนี้ 
with open('IMDB_tokenizer.pickle', 'rb') as handle:
    tk = pickle.load(handle)

การเตรียมข้อมูล Train และ Test ข้อมูลที่ X ที่นำมาใช้คือ sents_pad ที่ผ่านการ Padding แล้ว ซึ่งมีความยาวเท่ากันหมด 300 คำ ส่วนค่า y คือ df.label คือเลข 
0 1 0 0 1 ... เป็นต้น โดยนำมาแบ่งเพื่อใช้ Train และ Test ในสัดส่วนที่เท่ากัน คือ 50%

In [150]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sents_pad, 
                                                    df.label, 
                                                    test_size=0.5,
                                                    random_state=1)

X_train.shape, X_test.shape

((25000, 300), (25000, 300))

การสร้างแมทริกซ์ (Embedding Matrix) ในกรณีที่ไม่ได้ใช้ Pre-trained model ใดๆ สามารถเข้าสู่การสร้าง model และ train ได้เลย แต่ในกรณีนี้ ยกตัวอย่างการใช้ Pre-trained model ดังนั้น จึงต้องสร้างแมทริกซ์แต่ละคำขึ้นมาก่อน หลักการคือสร้างแมทริกซ์ขนาด vocab_size x embed_size (125495 x 100) โดย 125495 คือ Unique คำที่ไม่ซ้ำกันใน Dataset ส่วน 100 ความยาวของเวกเตอร์

In [151]:
#ขนาดของเวกเตอร์คือ 100
embed_size = wv.vector_size 

embedding_matrix = np.zeros((vocab_size, embed_size))
for word, i in tk.word_index.items():
    if i >= vocab_size:
        continue
    try:
        embedding_vector = wv[word]
        embedding_matrix[i] = embedding_vector[:embed_size]
    except KeyError:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), embed_size)

embedding_matrix.shape

(126512, 100)

In [152]:
#ดูข้อมูล matrix บางส่วน 
embedding_matrix[1][:2]

array([-0.038194  , -0.24487001])

การสร้าง Model จะต้องมีการกำหนดค่า vocab_size, embed_size และ weigths ก่อนหน้านี้ 

In [153]:
from keras.layers import GRU, Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, embed_size,
                    weights=[embedding_matrix], #Weight ใช้ Embedding matrix
                    trainable=True,
                    input_shape=(X_train.shape[1],)))

In [154]:
model.add(Bidirectional(GRU(units=128, activation='tanh')))
model.add(Dense(units=1, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 300, 100)          12651200  
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              176640    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 12,828,097
Trainable params: 12,828,097
Non-trainable params: 0
_________________________________________________________________


In [155]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [156]:
history = model.fit(X_train, y_train, 
                    epochs=5, batch_size=128,
                    verbose=1, validation_data=(X_test, y_test))

Epoch 1/5
196/196 [==============================] - 414s 2s/step - loss: 0.5233 - accuracy: 0.7268 - val_loss: 0.3384 - val_accuracy: 0.8582
Epoch 2/5
196/196 [==============================] - 435s 2s/step - loss: 0.2763 - accuracy: 0.8889 - val_loss: 0.2781 - val_accuracy: 0.8846
Epoch 3/5
196/196 [==============================] - 430s 2s/step - loss: 0.1641 - accuracy: 0.9418 - val_loss: 0.2979 - val_accuracy: 0.8872
Epoch 4/5
196/196 [==============================] - 436s 2s/step - loss: 0.0802 - accuracy: 0.9747 - val_loss: 0.3647 - val_accuracy: 0.8812
Epoch 5/5
196/196 [==============================] - 438s 2s/step - loss: 0.0354 - accuracy: 0.9902 - val_loss: 0.5013 - val_accuracy: 0.8747


In [157]:
#การประเมิน model 
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy: {:.3f}'.format(accuracy))

782/782 [==============================] - 92s 117ms/step - loss: 0.5013 - accuracy: 0.8747
Test accuracy: 0.875


ในกรณีที่ล็อกไม่ให้ Train ชั้น Embedding ให้กำหนด trainable=False ค่าพารามิเตอร์จะลดลง เช่น 12 ล้านเศษ เหลือ Trainable 176,897 ดังนี้ 

In [158]:
# model = Sequential()
# model.add(Embedding(vocab_size, embed_size,
#                     weights=[embedding_matrix], #Weight ใช้ Embedding matrix
#                     trainable=False, #เปลี่ยนจาก True เป็น False
#                     input_shape=(X_train.shape[1],)))

# model.summary()

การพยากรณ์ (Predict) การใช้งานคือการ input ข้อความ Review เข้าสู่ Model ให้ Predict บอกว่า ข้อความ Review มีอารมณ์ความรู้สึกในแง่บวกหรือลบ ข้อสำคัญคือจะต้องเข้ารหัส Tokenize ตัวเดียวกับที่ใช้ Train model 

In [159]:
from keras.preprocessing.text import text_to_word_sequence

comment1 = 'This film was about politics. I do not like ti. It sucks. Can I get money back?'

#ตัดคำ
words_tk = text_to_word_sequence(comment1)
print(words_tk)


['this', 'film', 'was', 'about', 'politics', 'i', 'do', 'not', 'like', 'ti', 'it', 'sucks', 'can', 'i', 'get', 'money', 'back']


การเข้ารหัส ใช้ tk ตัวเดียวกับที่เตรียมข้อมูลสำหรับการ Train

In [160]:
#การเข้ารหัส ใช้ tk ตัวเดียวกับที่เตรียมข้อมูลสำหรับการ Train
new_words_enc = tk.texts_to_sequences([comment1])
new_words_enc

[[10, 18, 12, 40, 2534, 9, 76, 20, 36, 32900, 8, 1693, 66, 9, 75, 290, 140]]

In [161]:
#ใส่ Padding ให้ความยาวเท่ากับที่ใช้ตอน Train Model 
X_new = pad_sequences(new_words_enc, truncating='post',
                      padding='post', maxlen=max_len)

X_new

array([[   10,    18,    12,    40,  2534,     9,    76,    20,    36,
        32900,     8,  1693,    66,     9,    75,   290,   140,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

เมื่อถอดรหัสคำ จะต้องกลับมาเหมือนประโยคต้นฉบับ แสดงว่าการถอดรหัสทำงานถูกต้อง

In [162]:
#เมื่อถอดรหัสคำ จะต้องกลับมาเหมือนประโยคต้นฉบับ แสดงว่าการถอดรหัสทำงานถูกต้อง
tk.sequences_to_texts(X_new)

['this film was about politics i do not like ti it sucks can i get money back']

ทำการ Predict ถ้าใได้ค่า Score 0.5 ขึ้นไป แสดงความว่าความรู้สึกเป็น Positive หมายถึงอารมณ์ความรู้สึกเป็นบวก แต่ถ้าค่าน้อยกว่า แสดงว่าเป็นแง่ลบ 

model.predict(X).apply(lambda p: 0 if p < .5 else 1)
#threshold = 0.5 
y_pred = model.predict(X_test)
ypred = (y_pred > 0.5).astype(int)
predicted = pd.DataFrame(ypred)
rename = predicted.rename(columns={0: 'China',
                                   1: 'United States'})
rename

In [163]:
result = model.predict(X_new)
print(np.squeeze(result).round(3))
print('Positive' if result[0] > 0.5 else 'Negative')

0.005
Negative


การ Predict หลาย Review โดยเพิ่ข้อความ Review แล้วรวมให้เป็น 1 list 

In [164]:
comment2 = 'this film was about politics. people talk about it. the film was good. i recommend'
reviews_new = [comment1, comment2]
for i in reviews_new:
    print(i)

This film was about politics. I do not like ti. It sucks. Can I get money back?
this film was about politics. people talk about it. the film was good. i recommend


ทำการ Padding แล้วส่งเข้า Model เพื่อ Predict จะได้ค่า Score ผลลัพธ์คือ Review คนแรก รู้สึกเชิงลบ ส่วน Review คนที่ 2 ความรู้สึกเชิงบวก 

In [165]:
new_words_enc = tk.texts_to_sequences(reviews_new)
X_new = pad_sequences(new_words_enc, truncating='post', 
                      padding='post', maxlen=max_len)
results = model.predict(X_new)
print(result.round(3))

[[0.005]]


In [166]:
#ทำนาย X_test 15 ต้วแรก 
result = model.predict(X_test[:15])
print(np.squeeze(result).round(2))

[0.   0.   0.19 0.   0.7  0.   0.89 0.   0.   0.   0.86 0.03 1.   1.
 0.  ]


In [167]:
#เปรียบเทียบกับค่าจริง y_test 
y_test[:15].tolist()

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0]